In [1]:
from pymongo import MongoClient
import pprint

import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time

In [2]:
# roster_urls = {}
# roster_urls['unc'] = 'https://goheels.com/sports/womens-basketball/roster/'
# roster_urls['duke'] = 'https://goduke.com/sports/womens-basketball/roster/'
# roster_urls['miami'] = 'https://hurricanesports.com/sports/womens-basketball/roster/'
# roster_urls['nc_state'] = 'https://gopack.com/sports/womens-basketball/roster/'

# #List of desired seasons of data - will combine with roster_urls to scrape
# seasons = ['2009-10', '2010-11', '2011-12', '2017-18', '2018-19', '2019-20']

In [3]:
#Grab requests and input into MongoDB - commented out after completing once
client = MongoClient('localhost', 27017)
db = client.wbb2
rosters = db.rosters

# for season in seasons:
#     for team, url in roster_urls.items():
#         r = requests.get(url + season)
#         rosters.insert_one({'team': team,
#                             'season': season,
#                             'html': r.text})
#         time.sleep(2)

At this point I have a cursor that contains all the HTML files containing each team's roster for every season.

Now I need to separate out the html but make sure to tie it to the correct team & season.

Fields I want are:
-heights
-names
-positions
-class_yr

Update MongoDB fields to have the keys with the fields you want with values that consist of the parsed out values.

*****Note*****
-Ran into an issue with parsing the data from Mongo.  I saved the html files in mongo using r.content.  R.content saves the files as bytes rather than strings (r.text).  Didn't find a quick solution to deal with bytes so pulled the html again into the 'wbb2' database and saved it using r.text.

--Update--
The data is fine for unc for the 2019-2020, 2018-19, 2017-2018 seasons, but I suspect the HTML changed so now I have to figure that out.  This wasn't apparent the first time because I was checking the oldest seasons as opposed to the last seasons.

-Iterate from the end of the cursor and once the lists fail to populate, use a different scheme for parsing.

In [4]:
cursor = rosters.find({})

In [5]:
seasons_2 = ['2017-18', '2018-19', '2019-20']
all_teams = []
for item in cursor:
    if item['season'] in seasons_2 or item['team'] == 'duke':
        soup = BeautifulSoup(item['html'], 'html.parser')
        #run parsing code
        heights = [item.text for item in soup.find_all(class_='height')]
        names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
        positions = [item.text for item in soup.find_all(class_='rp_position_short')]
        class_yr = [item.text for item in soup.find_all(class_='roster_class')]
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})
    else:
        soup = BeautifulSoup(item['html'], 'html.parser')
        #run parsing code
        heights = [item.text for item in soup.find_all(class_='sidearm-roster-player-height')]
        first_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-first-name')]
        last_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-last-name')]
        positions = [item.text for item in soup.find_all(class_='sidearm-list-card-details-item sidearm-roster-player-position-short')]
        class_yr = [item.text for item in soup.find_all(class_='sidearm-roster-player-academic-year')]
        
        #code to get first & last names
        names = [item[0] + ' ' + item[1] for item in zip(first_names, last_names)]
        
        #removes duplicate heights
        heights = heights[:len(names)]
        
        #standardize height format
        heights = [item.replace("'","-") for item in heights]
        heights = [item.replace('"', "") for item in heights]
        
        #remove duplicate class years
        class_yr = class_yr[-len(names):]
        
        #code to only keep characters relating to positions
        fixed_pos = []
        for string in positions:
                new_pos = [letter for letter in string if letter.isupper()]
                if len(new_pos) == 1:
                    fixed_pos.append(new_pos[0])
                else:
                    fixed_pos.append(new_pos[0] + "/" + new_pos[1])
        positions = fixed_pos
    
        #append desired fields to all_teams list
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})

Need to verify all records were correctly pulled by checking for blanks and storing the teams & indices.

In [6]:
# soup = BeautifulSoup(cursor[0]['html'], 'html.parser')
# print(soup.prettify())

In [7]:
blank_teams = []
blank_indices = []
i=0
for item in all_teams:
    if item['heights'] == []:
        blank_teams.append({'team': item['team'],
                            'season': item['season']
        })
        blank_indices.append(i)
        i+=1
    else:
        i+=1

print(blank_indices)
blank_teams

[]


[]

In [8]:
all_teams

[{'team': 'unc',
  'season': '2009-10',
  'heights': ['6-1',
   '5-5',
   '5-10',
   '5-11',
   '5-10',
   '6-5',
   '6-0',
   '5-6',
   '6-2',
   '6-6',
   '6-1',
   '6-4',
   '5-10',
   '5-8',
   '6-3'],
  'names': ['Nyree Williams',
   "She'la White",
   'Trinity Bursey',
   'Candace Wood',
   'Nicole Powell',
   'Chay Shegog',
   'Krista Gross',
   'Cetera DeGraffenreid',
   'Martina Wood',
   'Waltiea Rolle',
   'Laura Broomfield',
   'Cierra Robertson-Warren',
   'Tierra Ruffin-Pratt',
   'Italee Lucas',
   'Jessica Breland'],
  'positions': ['F',
   'G',
   'F',
   'G',
   'G',
   'F/C',
   'G',
   'G',
   'F/C',
   'F/C',
   'F',
   'F',
   'G',
   'G',
   'F'],
  'class_yr': ['Fr.',
   'So.',
   'Sr.',
   'Rs.',
   'Jr.',
   'So.',
   'Fr.',
   'Jr.',
   'Sr.',
   'Fr.',
   'So.',
   'Fr.',
   'Fr.',
   'Jr.',
   'Sr.']},
 {'team': 'duke',
  'season': '2009-10',
  'heights': ['5-8',
   '5-9',
   '5-9',
   '6-0',
   '6-0',
   '6-1',
   '6-2',
   '6-0',
   '6-1',
   '6-4',
   '6